In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

In [ ]:
COLUMNA_GRD_INTERNO = [
    "rut",
    "RUT_LIMPIO",
    "sexo_cod",
    "prevision_cod",
    "hospital_de_procedencia_cod",
    "fecha_ingreso",
    "fecha_de_egreso",
    "ano_de_egreso",
    "estancias_totales",
    "diag_01_principal_cod",
    "diag_01_principal_des",
    "diag_02_secundario_cod",
    "diag_02_secundario_des",
    "diag_03_secundario_cod",
    "diag_03_secundario_des",
    "proced_01_principal_cod",
    "proced_01_principal_des",
    "1ra_intervencion_cirujano",
    "especialidad_medica_de_la_intervencion_des",
]

RUTA_GRD_INTERNO = "../data/processed/grd_interno.csv"
COLUMNAS_FECHA_GRD_INT = ["fecha_ingreso", "fecha_de_egreso"]

In [ ]:
# Carga la base de datos de ambulatorio, eliminando 2023 debido a falla en registro
df_his = pd.read_csv("../data/external/his_procesada.csv")
df_his = df_his.query("ano != 2023")
df_his["id_paciente"] = df_his["id_paciente"].astype(str)

In [ ]:
# Carga la base de datos de Procedimientos - Datos Estadisticos
procedimientos = pd.read_csv("../data/external/procedimientos_procesada.csv")
procedimientos["id_paciente"] = procedimientos["id_paciente"].astype(str)

In [ ]:
# Une la base de HIS y Procedimientos (Datos estadisticos)
his_y_procedimientos = df_his.merge(procedimientos, how="left", on="id_paciente")

In [ ]:
# Obtiene los procedimientos de fibrobroncoscopia
PROCEDIMIENTOS_FIBRO = [
    "LARINGOTRAQUEOBRONCOSCOPIA CON FIBROSCOPIO",
    "NASOFARINGOLARINGOFIBROSCOPIA",
]

In [ ]:
# Define los registros que si tienen una fibrobroncoscopia
pacientes_con_fibrobronco = his_y_procedimientos.query("glosa.isin(@PROCEDIMIENTOS_FIBRO)")

In [ ]:
# Obtiene los pacientes que requieren una fibrobronco por diagnostico
pacientes_con_fibro_por_diag = pacientes_con_fibrobronco.groupby(
    ["codigo_diagnostico", "nombre_diagnostico"]
)["id_paciente"].nunique()
diagnosticos_que_requieren_fibro = pacientes_con_fibro_por_diag.index.get_level_values(0)

In [ ]:
# Obtiene el volumen total de pacientes para los diagnosticos que han tenido una fibrobronco
registros_totales_diags_de_fibro = (
    his_y_procedimientos.query("codigo_diagnostico.isin(@diagnosticos_que_requieren_fibro)")
).copy()

pacientes_totales_diags_de_fibro = registros_totales_diags_de_fibro.groupby(
    ["codigo_diagnostico", "nombre_diagnostico"]
)["id_paciente"].nunique()

In [ ]:
# Compara la cantidad de pacientes totales por diag y los que requieren fibrobronco
comparacion_pacientes_fibro = pd.concat(
    [pacientes_totales_diags_de_fibro, pacientes_con_fibro_por_diag], axis=1
)
comparacion_pacientes_fibro.columns = ["pacientes_totales_diag", "pacientes_que_requieren_fibro"]
comparacion_pacientes_fibro["porcentaje_fibro"] = (
    comparacion_pacientes_fibro["pacientes_que_requieren_fibro"]
    / comparacion_pacientes_fibro["pacientes_totales_diag"]
)

In [ ]:
comparacion_pacientes_fibro.sort_values(["pacientes_que_requieren_fibro", "porcentaje_fibro"], ascending=False).to_excel("fibro.xlsx")

In [ ]:
n_pacientes_totales_diags_de_fibro = registros_totales_diags_de_fibro["id_paciente"].nunique()
n_pacientes_que_requieren_fibro = pacientes_con_fibrobronco["id_paciente"].nunique()
porcentaje_fibro_por_diag = n_pacientes_que_requieren_fibro / n_pacientes_totales_diags_de_fibro

In [ ]:
print(f"Hay {n_pacientes_totales_diags_de_fibro} pacientes en los diags que requieren fibro")
print(f"Hay {n_pacientes_que_requieren_fibro} pacientes que requieren fibro")
print(f"% de Fibro por diagnostico: {porcentaje_fibro_por_diag:.2%}")

## Calculo de Desempeño

En este apartado, se quiere saber cuantos procedimientos son necesarios hacer por paciente. Para esto, se calcularán medidas de distribución

In [ ]:
conteo_paciente_procedimientos = (
    procedimientos.groupby("glosa")["id_paciente"].value_counts().reset_index()
)
desempeno_procedimientos = conteo_paciente_procedimientos.groupby("glosa")["count"].describe()